In [1]:
from getpass import getpass
#import arcgis
import pandas as pd
from arcgis.gis import GIS
#from arcgis.features import FeatureLayer
from arcgis.geometry import Polygon, filters #, project, intersect, areas_and_lengths
from mapping import get_layer, get_area_of_interest, get_aoi_feature_areas #, get_landcover_percentages


/opt/homebrew/Caskroom/miniforge/base/envs/mapping/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Authenticate with ArcGIS Online
username = getpass("Enter ArcGIS username: ")

password = getpass("Enter ArcGIS password: ")

gis = GIS("https://www.arcgis.com", username, password)

#### Get Webmap and layers

Reference map from ArcGIS which utilizes the Land Use/Land Cover of New Jersey 2015 ArcGIS Layer from NJDEP Bureau of GIS.

URL for Layer information: https://mapsdep.nj.gov/arcgis/rest/services/Features/Land_lu/MapServer/13

**Credit/Disclaimer:** This map was developed using New Jersey Department of Environmental Protection Geographic Information System digital data, but this secondary product has not been verified by NJDEP and is not state-authorized or endorsed.

Web map ID for map utilizign the above layer: https://www.arcgis.com/home/search.html?restrict=false&sortField=relevance&sortOrder=desc&searchTerm=2705228b2b154d0a906ef7a54e533fac#content

_Powered by Esri._

**Source:** Esri, NASA, NGA, USGS, FEMA | New Jersey Office of GIS, Esri, HERE, Garmin, SafeGraph, GeoTechnologies, Inc, MÈTI/NASA, USGS, EPA, NPS, US Census Bureau, USDA | Ni Department of Environmental Protection (NJDEP), Division of Information Technology (DOIT), Bureau of Geographic Information System (BGiS)


In [3]:
layer = get_layer(gis)

In [39]:
# Web map ID fo https://www.arcgis.com/home/search.html?restrict=false&sortField=relevance&sortOrder=desc&searchTerm=2705228b2b154d0a906ef7a54e533fac#content
'''WEBMAP_ID = "2705228b2b154d0a906ef7a54e533fac"

# Access the webmap item by ID and get operational layers from the data
webmap_item = gis.content.get(WEBMAP_ID)
webmap_data = webmap_item.get_data()
operational_layers = webmap_data["operationalLayers"]

# URL of the operational layer
layer_url = operational_layers[0].get('url')

# Access the layer
layer = FeatureLayer(layer_url)'''

In [41]:
layer.properties.capabilities

'Map,Query,Data'

We can't extract it for offline calculations.

#### Pull in dataset with the lat-long boxes for image taking regions.

In [4]:
df = pd.read_csv("satRgbFinal4.csv")
df.head()

,sat_label,sat_url,sat_Location,sat_Type,sat_Lat-Center,sat_Lon-Center,sat_Lat-NW,sat_Lon-NW,sat_Lat-SE,sat_Lon-SE,...,sat_cl3blue,sat_cl4blue,sat_cl1size,sat_cl2size,sat_cl3size,sat_cl4size,sat_cl1name,sat_cl2name,sat_cl3name,sat_cl4name
0,farmCherry,https://github.com/Imageomics/Andromeda/raw/ma...,Cherry Orchid Farm,farm,40.322965,-74.728754,40.332624,-74.749758,40.313732,-74.707701,...,0.128496,0.515794,0.317410,0.315147,0.297610,0.069834,gray24,darkolivegreen,gray14,gray57
1,farmDblBrook,https://github.com/Imageomics/Andromeda/raw/ma...,Double Brook Farm; Bayberry,farm,40.354852,-74.741672,40.365821,-74.762890,40.345643,-74.720636,...,0.279687,0.498397,0.390455,0.282155,0.264181,0.063210,gray23,gray12,darkolivegreen,ivory4
2,farmLawrenceville,https://github.com/Imageomics/Andromeda/raw/ma...,Lawrenceville School’s Big Red Farm,farm,40.318704,-74.744964,40.328533,-74.765787,40.309690,-74.723841,...,0.122767,0.476703,0.407344,0.302907,0.233696,0.056053,darkolivegreen,gray23,gray13,lightyellow4
3,farmStony,https://github.com/Imageomics/Andromeda/raw/ma...,Stony Ford Field Station Farm,farm,40.352764,-74.719865,40.363028,-74.740570,40.342669,-74.698661,...,0.278994,0.528382,0.402267,0.323160,0.224271,0.050302,gray22,gray12,darkolivegreen,gray56
4,gardenMtLake,https://github.com/Imageomics/Andromeda/raw/ma...,Mt. Lake,garden,40.365934,-74.672103,40.375474,-74.693112,40.356519,-74.650875,...,0.384347,0.616343,0.411457,0.288523,0.212111,0.087909,gray28,gray18,gray42,gray62


### Compile region landcover percentages for tilings of the satellite locations covering up to a half-mile around.

1. Compile list of areas of interest based on the identified locations
2. Check for all regions defined within 1/2 a mile of our identified satellite locations to compile master list.
3. Bin these regions into {Grassy, Dense Wood, Woody, Suburban, Watery, Urban, Agricultural}, as defined in [Sat Dictionary](https://github.com/Imageomics/Andromeda/blob/main/datasets/satelliteData/satRgbFinal4_dictionary.csv).
4. Area of interest is then set as 1-mi and 1/2-mi boxes located at each of the four corners of the satellite images, the middle of each side, and the center.
5. Calculate the percentage of each type of land cover (Grassy, Dense Wood, Woody, Suburban, Watery, Urban, Agricultural) present within each area of interest defined in #4.
6. Export this information (land cover percentage with associated lat-long values (centers and corners)) to CSV for merging with QUEST data.

In [4]:
# Fixed distance for boxes center on image lat/lon
BOUNDING_RADIUS = 0.5
# Convert distance to degrees (approx)
LAT_BOUNDING_RADIUS_DEG = BOUNDING_RADIUS / 69.0 # 69 miles per degree at 38 degrees North Latitude
LON_BOUNDING_RADIUS_DEG = BOUNDING_RADIUS / 54.6 # 54.6 miles per degree at 38 degrees North Latitude for longitudinal directions

##### 1. Compile list of areas of interest based on the identified locations

In [5]:
areas_oi = []
for sat_label in df.sat_label.values:
    temp = df.loc[df.sat_label == sat_label]
    lat_NW = temp['sat_Lat-NW'].values[0] + LAT_BOUNDING_RADIUS_DEG
    lon_NW = temp['sat_Lon-NW'].values[0] - LON_BOUNDING_RADIUS_DEG # min long
    lat_SE = temp['sat_Lat-SE'].values[0] - LAT_BOUNDING_RADIUS_DEG # min lat
    lon_SE = temp['sat_Lon-SE'].values[0] + LON_BOUNDING_RADIUS_DEG
    aoi_temp = Polygon({
                    "rings": [[[lon_NW, lat_SE], #min_lon, min_lat
                            [lon_NW, lat_NW],  #min_lon, max_lat
                            [lon_SE, lat_NW],  #max_lon, max_lat
                            [lon_SE, lat_SE],  #max_lon, min_lat
                            [lon_NW, lat_SE]]], #min_lon, min_lat
                    "spatialReference": {"wkid": 4326}
                })
    areas_oi.append(aoi_temp)

In [6]:
len(areas_oi)

18

##### 2. Check for all regions defined within 1/2 a mile of our identified satellite locations to compile master list. 
Do some pre-binning based on the layer's bins to fascilitate our classification.

In [7]:
regions_in_agro = set()
regions_in_barren = set()
regions_in_forest = set()
regions_in_water = set()
regions_in_urban = set()
wetlands = set()

for aoi in areas_oi:
    # Query the layer for features that intersect with the area of interest
    feature_set = layer.query(geometry_filter = filters.intersects(aoi))
    for feature in feature_set.features:
        region_label = feature.attributes['LABEL15']
        type_label = feature.attributes['TYPE15']
        if type_label == 'AGRICULTURE':
            regions_in_agro.add(region_label)
        elif type_label == 'BARREN LAND':
            regions_in_barren.add(region_label)
        elif type_label == 'FOREST':
            regions_in_forest.add(region_label)
        elif type_label == 'WATER':
            regions_in_water.add(region_label)
        elif type_label == 'URBAN':
            regions_in_urban.add(region_label)
        else:
            wetlands.add(region_label)

In [8]:
regions_in_agro

{'CROPLAND AND PASTURELAND',
 'ORCHARDS/VINEYARDS/NURSERIES/HORTICULTURAL AREAS',
 'OTHER AGRICULTURE'}

In [9]:
regions_in_barren

{'ALTERED LANDS', 'EXTRACTIVE MINING', 'TRANSITIONAL AREAS'}

In [10]:
regions_in_forest

{'CONIFEROUS BRUSH/SHRUBLAND',
 'CONIFEROUS FOREST (10-50% CROWN CLOSURE)',
 'CONIFEROUS FOREST (>50% CROWN CLOSURE)',
 'DECIDUOUS BRUSH/SHRUBLAND',
 'DECIDUOUS FOREST (10-50% CROWN CLOSURE)',
 'DECIDUOUS FOREST (>50% CROWN CLOSURE)',
 'MIXED DECIDUOUS/CONIFEROUS BRUSH/SHRUBLAND',
 'MIXED FOREST (>50% CONIFEROUS WITH 10-50% CROWN CLOSURE)',
 'MIXED FOREST (>50% CONIFEROUS WITH >50% CROWN CLOSURE)',
 'MIXED FOREST (>50% DECIDUOUS WITH 10-50% CROWN CLOSURE)',
 'MIXED FOREST (>50% DECIDUOUS WITH >50% CROWN CLOSURE)',
 'OLD FIELD (< 25% BRUSH COVERED)',
 'PLANTATION'}

In [11]:
regions_in_water

{'ARTIFICIAL LAKES',
 'BRIDGE OVER WATER',
 'EXPOSED FLATS',
 'NATURAL LAKES',
 'STREAMS AND CANALS'}

In [12]:
regions_in_urban

{'AIRPORT FACILITIES',
 'ATHLETIC FIELDS (SCHOOLS)',
 'CEMETERY',
 'COMMERCIAL/SERVICES',
 'INDUSTRIAL',
 'MAJOR ROADWAY',
 'MIXED RESIDENTIAL',
 'MIXED TRANSPORTATION CORRIDOR OVERLAP AREA',
 'MIXED URBAN OR BUILT-UP LAND',
 'OTHER URBAN OR BUILT-UP LAND',
 'RAILROADS',
 'RECREATIONAL LAND',
 'RESIDENTIAL, HIGH DENSITY OR MULTIPLE DWELLING',
 'RESIDENTIAL, RURAL, SINGLE UNIT',
 'RESIDENTIAL, SINGLE UNIT, LOW DENSITY',
 'RESIDENTIAL, SINGLE UNIT, MEDIUM DENSITY',
 'STADIUM, THEATERS, CULTURAL CENTERS AND ZOOS',
 'STORMWATER BASIN',
 'TRANSPORTATION/COMMUNICATION/UTILITIES',
 'UPLAND RIGHTS-OF-WAY DEVELOPED',
 'UPLAND RIGHTS-OF-WAY UNDEVELOPED'}

In [13]:
wetlands

{'AGRICULTURAL WETLANDS (MODIFIED)',
 'CONIFEROUS SCRUB/SHRUB WETLANDS',
 'CONIFEROUS WOODED WETLANDS',
 'DECIDUOUS SCRUB/SHRUB WETLANDS',
 'DECIDUOUS WOODED WETLANDS',
 'DISTURBED WETLANDS (MODIFIED)',
 'FORMER AGRICULTURAL WETLAND (BECOMING SHRUBBY, NOT BUILT-UP)',
 'HERBACEOUS WETLANDS',
 'MANAGED WETLAND IN BUILT-UP MAINTAINED REC AREA',
 'MANAGED WETLAND IN MAINTAINED LAWN GREENSPACE',
 'MIXED SCRUB/SHRUB WETLANDS (CONIFEROUS DOM.)',
 'MIXED SCRUB/SHRUB WETLANDS (DECIDUOUS DOM.)',
 'MIXED WOODED WETLANDS (DECIDUOUS DOM.)',
 'PHRAGMITES DOMINATE INTERIOR WETLANDS',
 'WETLAND RIGHTS-OF-WAY'}

##### 3. Bin these regions into {Grassy, Dense Wood, Woody, Suburban, Watery, Urban, Agricultural}, as defined in comments below and in [Landcover Dictionary]().

In [5]:
## Region bins: Grassy, Dense Wood, Woody, Suburban, Watery, Urban, Agricultural
# Grassy areas (pastures and school yards as well as golf courses):
GRASSY = ['ATHLETIC FIELDS (SCHOOLS)',  
            'CEMETERY',
            'CROPLAND AND PASTURELAND',
            'OLD FIELD (< 25% BRUSH COVERED)',
            'RECREATIONAL LAND'
            ]
# Dense woody areas (forests that are hardwood [deciduous] and soft wood [coniferous] and mixed:
DENSE_WOOD = ['CONIFEROUS FOREST (>50% CROWN CLOSURE)',
                'DECIDUOUS FOREST (>50% CROWN CLOSURE)',
                'MIXED FOREST (>50% CONIFEROUS WITH >50% CROWN CLOSURE)',
                'MIXED FOREST (>50% DECIDUOUS WITH >50% CROWN CLOSURE)'
                ]
# Woody areas (< 50% tree cover and shrublands where grass shows through but still lots of tall or short trees:
WOODY = ['CONIFEROUS BRUSH/SHRUBLAND',
        'CONIFEROUS FOREST (10-50% CROWN CLOSURE)',  
        'DECIDUOUS BRUSH/SHRUBLAND',
        'DECIDUOUS FOREST (10-50% CROWN CLOSURE)',
        'MIXED DECIDUOUS/CONIFEROUS BRUSH/SHRUBLAND',
        'MIXED FOREST (>50% CONIFEROUS WITH 10-50% CROWN CLOSURE)',
        'MIXED FOREST (>50% DECIDUOUS WITH 10-50% CROWN CLOSURE)'
        ]
# Suburban lawns (grassy areas with houses and gardens full of potential flowers; don’t need single or multiple dwelling differentiation, though):
SUBURBAN = ['MIXED RESIDENTIAL',
            'RESIDENTIAL, RURAL, SINGLE UNIT',
            'RESIDENTIAL, SINGLE UNIT, LOW DENSITY',
            'RESIDENTIAL, SINGLE UNIT, MEDIUM DENSITY'
            ]
# Watery areas and their associated wild areas that will contain some riverine grass and trees:
WATERY = ['AGRICULTURAL WETLANDS (MODIFIED)',
            'ARTIFICIAL LAKES',
            'CONIFEROUS SCRUB/SHRUB WETLANDS', 
            'CONIFEROUS WOODED WETLANDS',
            'DECIDUOUS SCRUB/SHRUB WETLANDS',
            'DECIDUOUS WOODED WETLANDS',
            'DISTURBED WETLANDS (MODIFIED)',
            'HERBACEOUS WETLANDS',
            'MANAGED WETLAND IN BUILT-UP MAINTAINED REC AREA',
            'MANAGED WETLAND IN MAINTAINED LAWN GREENSPACE',
            'MIXED SCRUB/SHRUB WETLANDS (CONIFEROUS DOM.)',
            'MIXED SCRUB/SHRUB WETLANDS (DECIDUOUS DOM.)',
            'MIXED WOODED WETLANDS (DECIDUOUS DOM.)',
            'NATURAL LAKES',
            'PHRAGMITES DOMINATE INTERIOR WETLANDS',
            'STORMWATER BASIN',
            'STREAMS AND CANALS'
            ]
# Areas with minimal natural habitats—urban areas (city centers, commercial sprawl, shopping centers, malls, 
# plenty of parking lots with gravel or blacktop (few potential flowers)).  
# This is a catch all category where flowers will be rare:
URBAN = ['AIRPORT FACILITIES',
        'ALTERED LANDS',
        'BRIDGE OVER WATER',
        'COMMERCIAL/SERVICES',
        'EXPOSED FLATS',
        'EXTRACTIVE MINING',
        'INDUSTRIAL',
        'MAJOR ROADWAY',
        'MIXED TRANSPORTATION CORRIDOR OVERLAP AREA',
        'MIXED URBAN OR BUILT-UP LAND',
        'OTHER URBAN OR BUILT-UP LAND',
        'RAILROADS',
        'RESIDENTIAL, HIGH DENSITY OR MULTIPLE DWELLING',
        'STADIUM, THEATERS, CULTURAL CENTERS AND ZOOS',
        'TRANSITIONAL AREAS',
        'TRANSPORTATION/COMMUNICATION/UTILITIES',
        'UPLAND RIGHTS-OF-WAY DEVELOPED',
        'UPLAND RIGHTS-OF-WAY UNDEVELOPED',
        'WETLAND RIGHTS-OF-WAY'
        ]
# Agricultural -- distinct from pasture, which should be grassy.
AGRICULTURAL = ['ORCHARDS/VINEYARDS/NURSERIES/HORTICULTURAL AREAS',
                'OTHER AGRICULTURE',
                'PLANTATION',
                'FORMER AGRICULTURAL WETLAND (BECOMING SHRUBBY, NOT BUILT-UP)']
# Region dictionary
REGION_DICT = {'GRASSY': GRASSY,
                'DENSE_WOOD': DENSE_WOOD,
                'WOODY': WOODY,
                'SUBURBAN': SUBURBAN,
                'WATERY': WATERY,
                'URBAN': URBAN,
                'AGRICULTURAL': AGRICULTURAL
}

##### 4. Set new reference points for new list of areas of interest (`local`: 1/2-mi box (.25 mi radius) and `broad`: 1-mi box (.5 mi radius)).

Make dictionary to retain reference to each location.

Reference points are determined as below, where 5 is the center of the satellite image:
```
1 --- 2 --- 3
|           |
4     5     6
|           |
7 --- 8 --- 9
```

In [30]:
def find_midpoint(x,y):
    z = x + y
    return z/2

In [6]:
ref_point_keys = []
lats = []
lons = []
for label in df.sat_label:
    temp = df.loc[df.sat_label == label]
    lat_NW = temp['sat_Lat-NW'].values[0]
    lon_NW = temp['sat_Lon-NW'].values[0] 
    lat_SE = temp['sat_Lat-SE'].values[0]
    lon_SE = temp['sat_Lon-SE'].values[0] 
    lat_center = temp['sat_Lat-Center'].values[0]
    lon_center = temp['sat_Lon-Center'].values[0]
    for i in range(1, 4):
        new_label = label + '_' + str(i)
        ref_point_keys.append(new_label)
        lats.append(lat_NW)
        if i == 1:
            lons.append(lon_NW)
        elif i == 2:
            lons.append(lon_center)
        else:
            lons.append(lon_SE)
    for i in range(4, 7):
        new_label = label + '_' + str(i)
        ref_point_keys.append(new_label)
        lats.append(lat_center)
        if i == 4:
            lons.append(lon_NW)
        elif i == 5:
            lons.append(lon_center)
        else:
            lons.append(lon_SE)
    for i in range(7, 10):
        new_label = label + '_' + str(i)
        ref_point_keys.append(new_label)
        lats.append(lat_SE)
        if i == 7:
            lons.append(lon_NW)
        elif i == 8:
            lons.append(lon_center)
        else:
            lons.append(lon_SE)

In [7]:
ref_point_keys

['farmCherry_1',
 'farmCherry_2',
 'farmCherry_3',
 'farmCherry_4',
 'farmCherry_5',
 'farmCherry_6',
 'farmCherry_7',
 'farmCherry_8',
 'farmCherry_9',
 'farmDblBrook_1',
 'farmDblBrook_2',
 'farmDblBrook_3',
 'farmDblBrook_4',
 'farmDblBrook_5',
 'farmDblBrook_6',
 'farmDblBrook_7',
 'farmDblBrook_8',
 'farmDblBrook_9',
 'farmLawrenceville_1',
 'farmLawrenceville_2',
 'farmLawrenceville_3',
 'farmLawrenceville_4',
 'farmLawrenceville_5',
 'farmLawrenceville_6',
 'farmLawrenceville_7',
 'farmLawrenceville_8',
 'farmLawrenceville_9',
 'farmStony_1',
 'farmStony_2',
 'farmStony_3',
 'farmStony_4',
 'farmStony_5',
 'farmStony_6',
 'farmStony_7',
 'farmStony_8',
 'farmStony_9',
 'gardenMtLake_1',
 'gardenMtLake_2',
 'gardenMtLake_3',
 'gardenMtLake_4',
 'gardenMtLake_5',
 'gardenMtLake_6',
 'gardenMtLake_7',
 'gardenMtLake_8',
 'gardenMtLake_9',
 'gardenPrinceton_1',
 'gardenPrinceton_2',
 'gardenPrinceton_3',
 'gardenPrinceton_4',
 'gardenPrinceton_5',
 'gardenPrinceton_6',
 'gardenPrinc

Reference points are determined as below, where 5 is the center of the satellite image:
```
1 --- 2 --- 3
|           |
4     5     6
|           |
7 --- 8 --- 9
```

In [8]:
ref_lat_lons = pd.DataFrame(data = {'ref_label': ref_point_keys, 'lat': lats, 'lon': lons})
ref_lat_lons

,ref_label,lat,lon
0,farmCherry_1,40.332624,-74.749758
1,farmCherry_2,40.332624,-74.728754
2,farmCherry_3,40.332624,-74.707701
3,farmCherry_4,40.322965,-74.749758
4,farmCherry_5,40.322965,-74.728754
...,...,...,...
166,urbanLibrary_5,40.351510,-74.660325
167,urbanLibrary_6,40.351510,-74.648996
168,urbanLibrary_7,40.346620,-74.671573
169,urbanLibrary_8,40.346620,-74.660325


In [9]:
# Regions: 'GRASSY', 'DENSE_WOOD', 'WOODY', 'SUBURBAN', 'WATERY', 'URBAN', 'AGRICULTURAL'

ref_lat_lons[['GRASSY', 'DENSE_WOOD', 'WOODY', 'SUBURBAN', 'WATERY', 'URBAN', 'AGRICULTURAL']] = 0
ref_lat_lons

,ref_label,lat,lon,GRASSY,DENSE_WOOD,WOODY,SUBURBAN,WATERY,URBAN,AGRICULTURAL
0,farmCherry_1,40.332624,-74.749758,0,0,0,0,0,0,0
1,farmCherry_2,40.332624,-74.728754,0,0,0,0,0,0,0
2,farmCherry_3,40.332624,-74.707701,0,0,0,0,0,0,0
3,farmCherry_4,40.322965,-74.749758,0,0,0,0,0,0,0
4,farmCherry_5,40.322965,-74.728754,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
166,urbanLibrary_5,40.351510,-74.660325,0,0,0,0,0,0,0
167,urbanLibrary_6,40.351510,-74.648996,0,0,0,0,0,0,0
168,urbanLibrary_7,40.346620,-74.671573,0,0,0,0,0,0,0
169,urbanLibrary_8,40.346620,-74.660325,0,0,0,0,0,0,0


We will need to insert a delay between each call to avoid server errors calling the ArcGIS API.

In [10]:
import time

Let's query the layer to get the local (1/2-mi box) landcover percentages.

In [11]:
local_landcover = ref_lat_lons.copy()

In [ ]:
aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == 'farmCherry_1', 'lat'].values[0], 
                            ref_lat_lons.loc[ref_lat_lons.ref_label == 'farmCherry_1', 'lon'].values[0],
                            .25)

feature_areas = get_aoi_feature_areas(layer, aoi)   

In [44]:
feature_areas

,Region_Label,Region,Intersect_Area,Region_Intersect_Total
0,RECREATIONAL LAND,GRASSY,15237.304035,18236.778042
1,STREAMS AND CANALS,WATERY,16763.158639,166535.806202
2,DECIDUOUS WOODED WETLANDS,WATERY,24365.101037,166535.806202
3,"RESIDENTIAL, RURAL, SINGLE UNIT",SUBURBAN,183.792205,16080.341177
4,"RESIDENTIAL, RURAL, SINGLE UNIT",SUBURBAN,15896.548972,16080.341177
5,OTHER URBAN OR BUILT-UP LAND,URBAN,2723.670945,6931.888719
6,OTHER URBAN OR BUILT-UP LAND,URBAN,4096.250455,6931.888719
7,OTHER URBAN OR BUILT-UP LAND,URBAN,111.967319,6931.888719
8,DECIDUOUS FOREST (10-50% CROWN CLOSURE),WOODY,99.906446,257219.870272
9,DECIDUOUS FOREST (10-50% CROWN CLOSURE),WOODY,13042.821468,257219.870272


In [12]:
def get_landcover_percentages(feature_areas):
    '''
    Function to determine percentage of area of interest covered by each land cover category (Grassy, Dense Wood, Woody, Suburban, Watery, Urban).
    
    Parameters:
    -----------
    layer - ArcGIS map layer to query.
    lat - Float. Latitude to center on.
    lon - Float. Longitude to center on.
    radius - Float. Fraction of a mile to use as side length for area of interest square.
    aoi - Area of interest. ArcGIS Polygon object.
    feature_areas - DataFrame containing regions and the area of their intersection with the given area of interest (lat/lon).

    Returns:
    --------
    region_percents - Dictionary of percentage of land in area of interest covered by each region category (index: Grassy, Dense Wood, Woody, Suburban, Watery, Urban).

    '''
    #aoi = get_area_of_interest(lat, lon, radius)
    #feature_areas = get_aoi_feature_areas(layer, aoi)
    total_area = 0
    region_percents = {'GRASSY': 0,
                    'DENSE_WOOD': 0,
                    'WOODY': 0,
                    'SUBURBAN': 0,
                    'WATERY': 0,
                    'URBAN': 0,
                    'AGRICULTURAL': 0
    }
    # Get total area of each region (Grassy, Dense Wood, Woody, Suburban, Watery, Urban) and sum total area
    for region in feature_areas.Region.unique():
        region_sum = feature_areas.loc[feature_areas.Region == region, 'Intersect_Area'].sum()
        feature_areas.loc[feature_areas.Region == region, 'Region_Intersect_Total'] = region_sum
        total_area = total_area + region_sum
    
    # Collect percentage of each region type 
    if total_area != 0:
        for region in feature_areas.Region.unique():
            region_sum = feature_areas.loc[feature_areas.Region == region, 'Region_Intersect_Total'].values[0]
            region_percents[region] = region_sum/total_area

    return region_percents


In [ ]:

aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == 'farmCherry_1', 'lat'].values[0], 
                            ref_lat_lons.loc[ref_lat_lons.ref_label == 'farmCherry_1', 'lon'].values[0],
                            .25)
print(aoi)
for _ in range(5):
    try:
        feature_areas = get_aoi_feature_areas(layer, aoi)
    except Exception as e:
        if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
            print(f"Error: {e}, retrying...")
            time.sleep(0.5)  # Wait for 0.5 seconds before retrying
        else:
            raise  # Raise the error if not 504 or 500


In [49]:
first_region_percents = get_landcover_percentages(feature_areas)

first_region_percents

{'GRASSY': 0.029124566886276234,
 'DENSE_WOOD': 0.25737649478202473,
 'WOODY': 0.4107862309272509,
 'SUBURBAN': 0.02568068608835093,
 'WATERY': 0.2659616306925609,
 'URBAN': 0.011070390623536288,
 'AGRICULTURAL': 0}

In [56]:
for key in first_region_percents.keys():
    local_landcover.loc[0, key] = first_region_percents[key]

In [ ]:
aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == 'farmCherry_2', 'lat'].values[0], 
                            ref_lat_lons.loc[ref_lat_lons.ref_label == 'farmCherry_2', 'lon'].values[0],
                            .25)
print(aoi)
for _ in range(5):
    try:
        feature_areas = get_aoi_feature_areas(layer, aoi)
    except Exception as e:
        if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
            print(f"Error: {e}, retrying...")
            time.sleep(0.5)  # Wait for 0.5 seconds before retrying
        else:
            raise  # Raise the error if not 504 or 500

region_percents = get_landcover_percentages(feature_areas)

In [59]:
second_region_percents = region_percents
for key in second_region_percents.keys():
    local_landcover.loc[1, key] = second_region_percents[key]
local_landcover

,ref_label,lat,lon,GRASSY,DENSE_WOOD,WOODY,SUBURBAN,WATERY,URBAN,AGRICULTURAL
0,farmCherry_1,40.332624,-74.749758,0.029125,0.257376,0.410786,0.025681,0.265962,0.011070,0.00000
1,farmCherry_2,40.332624,-74.728754,0.211565,0.089366,0.027461,0.076313,0.191876,0.121909,0.28151
2,farmCherry_3,40.332624,-74.707701,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,farmCherry_4,40.322965,-74.749758,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,farmCherry_5,40.322965,-74.728754,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...
157,univSolarFarm_5,40.334348,-74.647146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
158,univSolarFarm_6,40.334348,-74.626075,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
159,univSolarFarm_7,40.325165,-74.668232,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
160,univSolarFarm_8,40.325165,-74.647146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


We have first two, lets try collecting some more.

In [71]:
for ref_label in ref_lat_lons.ref_label[2:7]:
    aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lat'].values[0], 
                                ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lon'].values[0],
                                .25)
    print(aoi)
    for _ in range(5):
        try:
            feature_areas = get_aoi_feature_areas(layer, aoi)
        except Exception as e:
            if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
                print(f"Error: {e}, retrying...")
                time.sleep(0.5)  # Wait for 0.5 seconds before retrying
            else:
                raise  # Raise the error if not 504 or 500

    region_percents = get_landcover_percentages(feature_areas)
    print(region_percents)
    for key in region_percents.keys():
            local_landcover.loc[local_landcover.ref_label == ref_label, key] = region_percents[key]


{'rings': [[[-74.71227975457876, 40.329000811594206], [-74.71227975457876, 40.3362471884058], [-74.70312224542124, 40.3362471884058], [-74.70312224542124, 40.329000811594206], [-74.71227975457876, 40.329000811594206]]], 'spatialReference': {'wkid': 4326}}
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
Error: Error computing areas and lengths
(Error Code: 500), retrying...
Error: Error computing areas and lengths
(Error Code: 500), retrying...
Error: Error executing intersect
(Error Code: 500), retrying...
{'GRASSY': 0.21438039986314447, 'DENSE_WOOD': 0.24300790754705442, 'WOODY': 0.12122722445898095, 'SUBURBAN': 0.0982786859509579, 'WATERY': 0.3204418641096262, 'URBAN': 0.0026639180702360665, 'AGRICULTURAL': 0}
{'rings': [[[-74.75433675457876, 40.31934181159421], [-74.75433675457876, 40.3265881884058], [-74.74517924542124, 40.3265881884058], [-74.74517924542124, 40.31934181159421], [-74.

In [73]:
local_landcover.head(10)

,ref_label,lat,lon,GRASSY,DENSE_WOOD,WOODY,SUBURBAN,WATERY,URBAN,AGRICULTURAL
0,farmCherry_1,40.332624,-74.749758,0.029125,0.257376,0.410786,0.025681,0.265962,0.011070,0.000000
1,farmCherry_2,40.332624,-74.728754,0.211565,0.089366,0.027461,0.076313,0.191876,0.121909,0.281510
2,farmCherry_3,40.332624,-74.707701,0.214380,0.243008,0.121227,0.098279,0.320442,0.002664,0.000000
3,farmCherry_4,40.322965,-74.749758,0.460249,0.269608,0.087560,0.009368,0.163003,0.010175,0.000037
4,farmCherry_5,40.322965,-74.728754,0.434925,0.077210,0.216792,0.094412,0.161405,0.000995,0.014261
5,farmCherry_6,40.322965,-74.707701,0.035446,0.102308,0.021004,0.008367,0.132534,0.700342,0.000000
6,farmCherry_7,40.313732,-74.749758,0.240040,0.127685,0.004271,0.000001,0.607626,0.020377,0.000000
7,farmCherry_8,40.313732,-74.728754,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,farmCherry_9,40.313732,-74.707701,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,farmDblBrook_1,40.365821,-74.762890,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Let's collect a larger batch now that we have a time estimate.

In [74]:
for ref_label in ref_lat_lons.ref_label[7:40]:
    aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lat'].values[0], 
                                ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lon'].values[0],
                                .25)
    print(aoi)
    for _ in range(5):
        try:
            feature_areas = get_aoi_feature_areas(layer, aoi)
        except Exception as e:
            if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
                print(f"Error: {e}, retrying...")
                time.sleep(0.5)  # Wait for 0.5 seconds before retrying
            else:
                raise  # Raise the error if not 504 or 500

    region_percents = get_landcover_percentages(feature_areas)
    print(region_percents)
    for key in region_percents.keys():
            local_landcover.loc[local_landcover.ref_label == ref_label, key] = region_percents[key]


{'rings': [[[-74.73333275457875, 40.310108811594205], [-74.73333275457875, 40.3173551884058], [-74.72417524542124, 40.3173551884058], [-74.72417524542124, 40.310108811594205], [-74.73333275457875, 40.310108811594205]]], 'spatialReference': {'wkid': 4326}}
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
Error: Error executing intersect
(Error Code: 500), retrying...
{'GRASSY': 0.22501192686792126, 'DENSE_WOOD': 0.08550352159056489, 'WOODY': 0.07722134715581534, 'SUBURBAN': 0.45897576948606944, 'WATERY': 0.1151611267047476, 'URBAN': 0.023948627592952677, 'AGRICULTURAL': 0.014177680601928639}
{'rings': 

We got a crash, output copied to `txt` file to determine how much we managed to get, and let's start on next batch.

In [12]:
for ref_label in ref_lat_lons.ref_label[40:60]:
    aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lat'].values[0], 
                                ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lon'].values[0],
                                .25)
    print(aoi)
    for _ in range(5):
        try:
            feature_areas = get_aoi_feature_areas(layer, aoi)
        except Exception as e:
            if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
                print(f"Error: {e}, retrying...")
                time.sleep(0.5)  # Wait for 0.5 seconds before retrying
            else:
                raise  # Raise the error if not 504 or 500

    region_percents = get_landcover_percentages(feature_areas)
    print(region_percents)
    for key in region_percents.keys():
            local_landcover.loc[local_landcover.ref_label == ref_label, key] = region_percents[key]


{'rings': [[[-74.67668175457877, 40.362310811594206], [-74.67668175457877, 40.3695571884058], [-74.66752424542125, 40.3695571884058], [-74.66752424542125, 40.362310811594206], [-74.67668175457877, 40.362310811594206]]], 'spatialReference': {'wkid': 4326}}
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
Error: Error executing intersect
(Error Code: 500), retrying...
{'GRASSY': 0.25366602566258606, 'DENSE_WOOD': 0.230422366878332, 'WOODY': 0.02879129964664462, 'SUBURBAN': 0.19889459040405413, 'WATERY': 0.16560136684156823, 'URBAN': 0.03888683993962864, 'AGRICULTURAL': 0.08373751062718623}
{'rings': [[[

In [14]:
local_landcover.head(60)

,ref_label,lat,lon,GRASSY,DENSE_WOOD,WOODY,SUBURBAN,WATERY,URBAN,AGRICULTURAL
0,farmCherry_1,40.332624,-74.749758,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,farmCherry_2,40.332624,-74.728754,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,farmCherry_3,40.332624,-74.707701,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,farmCherry_4,40.322965,-74.749758,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,farmCherry_5,40.322965,-74.728754,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,farmCherry_6,40.322965,-74.707701,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,farmCherry_7,40.313732,-74.749758,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,farmCherry_8,40.313732,-74.728754,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,farmCherry_9,40.313732,-74.707701,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,farmDblBrook_1,40.365821,-74.762890,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Save what we got to CSV file

In [15]:
local_landcover.to_csv("../local_40-60.csv", index = False)

Next batch

In [12]:
for ref_label in ref_lat_lons.ref_label[60:80]:
    aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lat'].values[0], 
                                ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lon'].values[0],
                                .25)
    print(aoi)
    for _ in range(5):
        try:
            feature_areas = get_aoi_feature_areas(layer, aoi)
        except Exception as e:
            if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
                print(f"Error: {e}, retrying...")
                time.sleep(0.5)  # Wait for 0.5 seconds before retrying
            else:
                raise  # Raise the error if not 504 or 500

    region_percents = get_landcover_percentages(feature_areas)
    print(region_percents)
    for key in region_percents.keys():
            local_landcover.loc[local_landcover.ref_label == ref_label, key] = region_percents[key]


{'rings': [[[-74.80910175457876, 40.3511478115942], [-74.80910175457876, 40.358394188405796], [-74.79994424542124, 40.358394188405796], [-74.79994424542124, 40.3511478115942], [-74.80910175457876, 40.3511478115942]]], 'spatialReference': {'wkid': 4326}}
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
Error: Error executing project
(Error Code: 500), retrying...
{'GRASSY': 0.006764540148500214, 'DENSE_WOOD': 0.21454989933124555, 'WOODY': 0.03759530266289749, 'SUBURBAN': 0, 'WATERY': 0.04434210255963118, 'URBAN': 0.6967481552977256, 'AGRICULTURAL': 0}
{'rings': [[[-74.78802075457875, 40.3511478115942], [-74.78802075457875, 40.358394188405796], [-74.77886324542123, 40.358394188405796], [-74.77886324542123, 40.3511478115942], [-74.78802075457875, 40.3511478115942]]], 'spatialReference': {'wkid': 4326}}
Error: Error computing areas and lengths
(Error Code: 500), retrying...
Error: Error comput

Save what we got to CSV

In [14]:
local_landcover.to_csv("local_landcover60-80.csv", index = False)

Next batch

In [15]:
for ref_label in ref_lat_lons.ref_label[80:90]:
    aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lat'].values[0], 
                                ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lon'].values[0],
                                .25)
    print(aoi)
    for _ in range(5):
        try:
            feature_areas = get_aoi_feature_areas(layer, aoi)
        except Exception as e:
            if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
                print(f"Error: {e}, retrying...")
                time.sleep(0.5)  # Wait for 0.5 seconds before retrying
            else:
                raise  # Raise the error if not 504 or 500

    region_percents = get_landcover_percentages(feature_areas)
    print(region_percents)
    for key in region_percents.keys():
            local_landcover.loc[local_landcover.ref_label == ref_label, key] = region_percents[key]


{'rings': [[[-74.65306675457876, 40.3453288115942], [-74.65306675457876, 40.352575188405794], [-74.64390924542124, 40.352575188405794], [-74.64390924542124, 40.3453288115942], [-74.65306675457876, 40.3453288115942]]], 'spatialReference': {'wkid': 4326}}
Error: Error computing areas and lengths
(Error Code: 500), retrying...
Error: Error executing project
(Error Code: 500), retrying...
Error: Error executing intersect
(Error Code: 500), retrying...
{'GRASSY': 0.1426897731482679, 'DENSE_WOOD': 0, 'WOODY': 0, 'SUBURBAN': 0.2623650818345933, 'WATERY': 0.00024122340304316624, 'URBAN': 0.5947039216140955, 'AGRICULTURAL': 0}
{'rings': [[[-74.76330575457875, 40.3690408115942], [-74.76330575457875, 40.3762871884058], [-74.75414824542123, 40.3762871884058], [-74.75414824542123, 40.3690408115942], [-74.76330575457875, 40.3690408115942]]], 'spatialReference': {'wkid': 4326}}
Error: Error executing project
(Error Code: 500), retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>

Save what we got to CSV, then next batch.

In [16]:
local_landcover.to_csv("local_landcover80-90.csv", index = False)

In [17]:
for ref_label in ref_lat_lons.ref_label[90:110]:
    aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lat'].values[0], 
                                ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lon'].values[0],
                                .25)
    print(aoi)
    for _ in range(5):
        try:
            feature_areas = get_aoi_feature_areas(layer, aoi)
        except Exception as e:
            if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
                print(f"Error: {e}, retrying...")
                time.sleep(0.5)  # Wait for 0.5 seconds before retrying
            else:
                raise  # Raise the error if not 504 or 500

    region_percents = get_landcover_percentages(feature_areas)
    print(region_percents)
    for key in region_percents.keys():
            local_landcover.loc[local_landcover.ref_label == ref_label, key] = region_percents[key]


{'rings': [[[-74.74513175457876, 40.360394811594205], [-74.74513175457876, 40.3676411884058], [-74.73597424542125, 40.3676411884058], [-74.73597424542125, 40.360394811594205], [-74.74513175457876, 40.360394811594205]]], 'spatialReference': {'wkid': 4326}}
Error: Error computing areas and lengths
(Error Code: 500), retrying...
Error: Error executing project
(Error Code: 500), retrying...
Error: Error computing areas and lengths
(Error Code: 500), retrying...
Error: Error computing areas and lengths
(Error Code: 500), retrying...
{'GRASSY': 0.08257434364183398, 'DENSE_WOOD': 0.2916005294605249, 'WOODY': 0.3328371241879725, 'SUBURBAN': 0.016263631904082008, 'WATERY': 0.1360465689405919, 'URBAN': 0.14067780186499465, 'AGRICULTURAL': 0}
{'rings': [[[-74.72419975457876, 40.360394811594205], [-74.72419975457876, 40.3676411884058], [-74.71504224542124, 40.3676411884058], [-74.71504224542124, 40.360394811594205], [-74.72419975457876, 40.360394811594205]]], 'spatialReference': {'wkid': 4326}}
Er

Save what we got to CSV, then next batch.

In [18]:
local_landcover.to_csv("local_landcover90-110.csv", index = False)

In [19]:
for ref_label in ref_lat_lons.ref_label[110:130]:
    aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lat'].values[0], 
                                ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lon'].values[0],
                                .25)
    print(aoi)
    for _ in range(5):
        try:
            feature_areas = get_aoi_feature_areas(layer, aoi)
        except Exception as e:
            if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
                print(f"Error: {e}, retrying...")
                time.sleep(0.5)  # Wait for 0.5 seconds before retrying
            else:
                raise  # Raise the error if not 504 or 500

    region_percents = get_landcover_percentages(feature_areas)
    print(region_percents)
    for key in region_percents.keys():
            local_landcover.loc[local_landcover.ref_label == ref_label, key] = region_percents[key]


{'rings': [[[-74.78801675457876, 40.3342168115942], [-74.78801675457876, 40.3414631884058], [-74.77885924542124, 40.3414631884058], [-74.77885924542124, 40.3342168115942], [-74.78801675457876, 40.3342168115942]]], 'spatialReference': {'wkid': 4326}}
Error: Error computing areas and lengths
(Error Code: 500), retrying...
Error: Error executing project
(Error Code: 500), retrying...
Error: Error executing project
(Error Code: 500), retrying...
Error: Error computing areas and lengths
(Error Code: 500), retrying...
{'GRASSY': 0.1676150815272561, 'DENSE_WOOD': 0.38384930763815106, 'WOODY': 0.06750636115555315, 'SUBURBAN': 0.21030085633156606, 'WATERY': 0.1381064539178113, 'URBAN': 0.03262193942966226, 'AGRICULTURAL': 0}
{'rings': [[[-74.83015375457876, 40.3246578115942], [-74.83015375457876, 40.331904188405794], [-74.82099624542124, 40.331904188405794], [-74.82099624542124, 40.3246578115942], [-74.83015375457876, 40.3246578115942]]], 'spatialReference': {'wkid': 4326}}
Error: <html>
<head>

Save what we got to CSV, then next batch.

In [20]:
local_landcover.to_csv("local_landcover90-130.csv", index = False)

In [21]:
for ref_label in ref_lat_lons.ref_label[130:150]:
    aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lat'].values[0], 
                                ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lon'].values[0],
                                .25)
    print(aoi)
    for _ in range(5):
        try:
            feature_areas = get_aoi_feature_areas(layer, aoi)
        except Exception as e:
            if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
                print(f"Error: {e}, retrying...")
                time.sleep(0.5)  # Wait for 0.5 seconds before retrying
            else:
                raise  # Raise the error if not 504 or 500

    region_percents = get_landcover_percentages(feature_areas)
    print(region_percents)
    for key in region_percents.keys():
            local_landcover.loc[local_landcover.ref_label == ref_label, key] = region_percents[key]


{'rings': [[[-74.66121775457876, 40.340350811594206], [-74.66121775457876, 40.3475971884058], [-74.65206024542124, 40.3475971884058], [-74.65206024542124, 40.340350811594206], [-74.66121775457876, 40.340350811594206]]], 'spatialReference': {'wkid': 4326}}
Error: Error computing areas and lengths
(Error Code: 500), retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
{'GRASSY': 0.19807613177806008, 'DENSE_WOOD': 0.025151219656032217, 'WOODY': 0.0027257749700444534, 'SUBURBAN': 0.004256564535340479, 'WATERY': 0.003953269101760108, 'URBAN': 0.7658370399587627, 'AGRICULTURAL': 0}
{'rings': [[[-74.64029475457876, 40.340350811594206], [-74.64029475457876, 40.3475971884058], [-74.63113724542124, 40.3475971884058], [-74.63113724542124, 40.340350

type: 'int' object is not subscriptable

Odd error, let's resume where it failed.

In [22]:
for ref_label in ref_lat_lons.ref_label[138:150]:
    aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lat'].values[0], 
                                ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lon'].values[0],
                                .25)
    print(aoi)
    for _ in range(5):
        try:
            feature_areas = get_aoi_feature_areas(layer, aoi)
        except Exception as e:
            if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
                print(f"Error: {e}, retrying...")
                time.sleep(0.5)  # Wait for 0.5 seconds before retrying
            else:
                raise  # Raise the error if not 504 or 500

    region_percents = get_landcover_percentages(feature_areas)
    print(region_percents)
    for key in region_percents.keys():
            local_landcover.loc[local_landcover.ref_label == ref_label, key] = region_percents[key]


{'rings': [[[-74.67591375457876, 40.3390938115942], [-74.67591375457876, 40.3463401884058], [-74.66675624542124, 40.3463401884058], [-74.66675624542124, 40.3390938115942], [-74.67591375457876, 40.3390938115942]]], 'spatialReference': {'wkid': 4326}}
Error: Error executing project
(Error Code: 500), retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
{'GRASSY': 0.07935525773185544, 'DENSE_WOOD': 0.12334798383761646, 'WOODY': 0.018138546081072165, 'SUBURBAN': 0.6481212035807139, 'WATERY': 0, 'URBAN': 0.13103700876874197, 'AGRICULTURAL': 0}
{'rings': [[[-74.65488375457876, 40.3390938115942], [-74.65488375457876, 40.3463401884058], [-74.64572624542124, 40.3463401884058], [-74.64572624542124, 40.3390938115942], [-74.65488375457876, 40.339093

In [24]:
local_landcover.tail(50)

,ref_label,lat,lon,GRASSY,DENSE_WOOD,WOODY,SUBURBAN,WATERY,URBAN,AGRICULTURAL
112,schHopeHS_TimberMS_5,40.328281,-74.804559,0.154794,0.089834,0.061515,0.533318,0.017065,0.039304,0.104169
113,schHopeHS_TimberMS_6,40.328281,-74.783438,0.016962,0.020883,0.069624,0.797935,0.035248,0.059348,0.000000
114,schHopeHS_TimberMS_7,40.319148,-74.825575,0.407431,0.137746,0.291865,0.083178,0.022311,0.039682,0.017788
115,schHopeHS_TimberMS_8,40.319148,-74.804559,0.150641,0.185235,0.226295,0.381080,0.032943,0.001839,0.021968
116,schHopeHS_TimberMS_9,40.319148,-74.783438,0.045971,0.180035,0.188913,0.533477,0.018411,0.027810,0.005383
117,schTollGram_1,40.330632,-74.809579,0.492667,0.050270,0.024761,0.188435,0.127036,0.091086,0.025745
118,schTollGram_2,40.330632,-74.788680,0.055931,0.074509,0.006665,0.671170,0.009128,0.166041,0.016556
119,schTollGram_3,40.330632,-74.767619,0.256790,0.030311,0.183005,0.147305,0.286330,0.000527,0.095732
120,schTollGram_4,40.321026,-74.809579,0.056638,0.129051,0.253045,0.485480,0.059586,0.000000,0.016200
121,schTollGram_5,40.321026,-74.788680,0.043401,0.048901,0.008395,0.825915,0.000854,0.072533,0.000000


Save what we got to CSV, then last batch.

In [25]:
local_landcover.to_csv("local_landcover90-150.csv", index = False)

In [26]:
for ref_label in ref_lat_lons.ref_label[150:]:
    aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lat'].values[0], 
                                ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lon'].values[0],
                                .25)
    print(aoi)
    for _ in range(5):
        try:
            feature_areas = get_aoi_feature_areas(layer, aoi)
        except Exception as e:
            if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
                print(f"Error: {e}, retrying...")
                time.sleep(0.5)  # Wait for 0.5 seconds before retrying
            else:
                raise  # Raise the error if not 504 or 500

    region_percents = get_landcover_percentages(feature_areas)
    print(region_percents)
    for key in region_percents.keys():
            local_landcover.loc[local_landcover.ref_label == ref_label, key] = region_percents[key]


{'rings': [[[-74.67978175457876, 40.3329558115942], [-74.67978175457876, 40.3402021884058], [-74.67062424542124, 40.3402021884058], [-74.67062424542124, 40.3329558115942], [-74.67978175457876, 40.3329558115942]]], 'spatialReference': {'wkid': 4326}}
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
{'GRASSY': 0.04959624649772135, 'DENSE_WOOD': 0.16949123028177754, 'WOODY': 0, 'SUBURBAN': 0.7483130987709091, 'WATERY': 0.013359527446300052, 'URBAN': 0.019239897003291772, 'AGRICULTURAL': 0}
{'rings': [[[-74.65881975457876, 40.3329558115942], [-74.65881975457876, 40.3402021884058], [-74.64966224542124, 40.3402021884058], [-74.64966224542124, 40.3329558115942], [-74.65881975457876, 40.3329558115942]]], 'spatialReference': {'wkid': 4326}}
Error: Error executing intersect
(Error Code: 500), retrying...
Error: Error executing intersect
(Error Code: 500), retrying...
Error: Error executing intersect

In [27]:
local_landcover.to_csv("local_landcover60-end.csv", index = False)

Finally, let's go back and get the three we missed when it crashed.

In [28]:
for ref_label in ref_lat_lons.ref_label[37:40]:
    aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lat'].values[0], 
                                ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lon'].values[0],
                                .25)
    print(aoi)
    for _ in range(5):
        try:
            feature_areas = get_aoi_feature_areas(layer, aoi)
        except Exception as e:
            if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
                print(f"Error: {e}, retrying...")
                time.sleep(0.5)  # Wait for 0.5 seconds before retrying
            else:
                raise  # Raise the error if not 504 or 500

    region_percents = get_landcover_percentages(feature_areas)
    print(region_percents)
    for key in region_percents.keys():
            local_landcover.loc[local_landcover.ref_label == ref_label, key] = region_percents[key]


{'rings': [[[-74.67668175457877, 40.3718508115942], [-74.67668175457877, 40.379097188405794], [-74.66752424542125, 40.379097188405794], [-74.66752424542125, 40.3718508115942], [-74.67668175457877, 40.3718508115942]]], 'spatialReference': {'wkid': 4326}}
Error: Error computing areas and lengths
(Error Code: 500), retrying...
Error: Error computing areas and lengths
(Error Code: 500), retrying...
Error: Error executing intersect
(Error Code: 500), retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
{'GRASSY': 0, 'DENSE_WOOD': 0.3176572991889506, 'WOODY': 0.009926547129799566, 'SUBURBAN': 0.5899630123100005, 'WATERY': 0.05848023272042829, 'URBAN': 0.023972908650820933, 'AGRICULTURAL': 0}
{'rings': [[[-74.65545375457876, 40.3718508115942], [-74.65545375457876, 40.379097188405794], [-74.64629624542124, 40.379097188405794], [-74.64629624542124, 40.3718508115942], [-74.65545375457876, 40

In [29]:
local_landcover.to_csv("local_landcover37-40,60-end.csv", index = False)

Files combined in `CSV_editing_local.ipynb`

Urban Library added to Satellite datapoints, so we will now collect the landcover percentages for it.

In [13]:
for ref_label in ref_lat_lons.ref_label[162:]:
    aoi = get_area_of_interest(ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lat'].values[0], 
                                ref_lat_lons.loc[ref_lat_lons.ref_label == ref_label, 'lon'].values[0],
                                .25)
    print(aoi)
    for _ in range(5):
        try:
            feature_areas = get_aoi_feature_areas(layer, aoi)
        except Exception as e:
            if '504' in str(e) or '500' in str(e):  # If the error is a 504/500 error
                print(f"Error: {e}, retrying...")
                time.sleep(0.5)  # Wait for 0.5 seconds before retrying
            else:
                raise  # Raise the error if not 504 or 500

    region_percents = get_landcover_percentages(feature_areas)
    print(region_percents)
    for key in region_percents.keys():
            local_landcover.loc[local_landcover.ref_label == ref_label, key] = region_percents[key]


{'rings': [[[-74.67615175457875, 40.3536358115942], [-74.67615175457875, 40.360882188405796], [-74.66699424542124, 40.360882188405796], [-74.66699424542124, 40.3536358115942], [-74.67615175457875, 40.3536358115942]]], 'spatialReference': {'wkid': 4326}}
Error: Error executing intersect
(Error Code: 500), retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
Error: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
, retrying...
{'GRASSY': 0.09833584634433351, 'DENSE_WOOD': 0.14608478602318295, 'WOODY': 0.010291684057015308, 'SUBURBAN': 0.4082144070966529, 'WATERY': 0.17845567176628752, 'URBAN': 0.08455074882271224, 'AGRICULTURAL': 0.07406685588981554}
{'rings': [[[

In [14]:
local_landcover.to_csv("local_landcover.csv", index = False)